# Plot positions

[direct link](#Display)

In [ ]:
# parameters

combined_data_file = "combined_positions.csv"
start_date = "2023-04-01T00:00:00"

In [ ]:
import pandas as pd
import hvplot.pandas
from bokeh.models import HoverTool
import geoviews as gv

## Read data

In [ ]:
df = pd.read_csv(combined_data_file, parse_dates=["Timestamp", ])
df = df.where(df.Timestamp > start_date).dropna()

## Plot

In [ ]:
hover = HoverTool(tooltips=[
    ("Platform", "@Platform"),
    ("Type", "@Type"),
    ("Timestamp", "@Timestamp"),
    ("Longitude", "@Longitude_"),  # display potentially redacted positions
    ("Latitude", "@Latitude_"),  # display potentially redacted positions
])

In [ ]:
df["Timestamp"] = df["Timestamp"].astype(str)  # necessary for proper display ?

In [ ]:
plot = df.hvplot.points(
    x="Longitude", y="Latitude", color="Platform", size=50,
    hover_cols=["Platform", "Type", "Longitude_", "Latitude_", "Timestamp", ], tools=[hover, ],
    geo=True, tiles="OSM",
    xlim=(9.6, 11.4), ylim=(54.2, 55),
    frame_width=500,
)

## Display

In [ ]:
display(plot)